# NLP[0] -- 语料和数据收集 -- 文本预处理及数据库文本分析

## MutBot这个项目设计包含自动收集数据的功能, 以QQ为主要用用户接口和数据来源, 本文为NLP的"第0步" -- 文本预处理和数据整理.  

本文涉及的内容:  
1. 用第三方库: `langid`检测输入语种, `jieba`做分词和词性标注  
2. 正则表达式re模块的简单运用  
3. SQLite基础数据库操作(创建/检查/写入和读取)
4. 第三方库`pandas`的介绍和简单运用(本例用于简化数据查询操作)
5. 有害内容及无意义内容的过滤清洗(这里使用cherry库)

In [4]:
import time  # 用于计时比较
import sys   # 本文中用于观察变量内存占用

本项目的文本数据来源主要为QQ群, 所以, 各种各样情况都可能遇到: 

In [73]:
测试文本1 = "[CQ:at,id=qq/user/15*****] 氖泡接在驱动电源负和地线之间[CQ:face,id=108][CQ:face,id=108][CQ:at,id=qq/user/15*****]"
测试文本2 = "牛啤"
测试文本3 = "ΕΞΖΝи△M"
测试文本4 = "This is a test text..."
测试文本5 = '<h2><a id="user-content-probability-normalization" class="anchor" aria-hidden="true" href="#probability-normalization"></a>Probability Normalization</h2>'
测试文本6 = "import win32event as event; event_tigger = event.CreateEvent(None, False, False, 'Global\\233__update'); event.SetEvent(event_tigger)"
测试文本7 = "http://yhhtech.cn"
测试文本8 = "2333锟斤拷"

# 为了方便后面演示, 将上面的单条文本转为列表
测试文本组 = []
for i in range(1, 9):
    _tmp_句子 = eval("测试文本%d" % i)
    if len(_tmp_句子) > 83:  # 忽略多余内容(其实主要是嫌显示不爽). 一般来说, 一段话的前80个字足够判断句子所使用的语种了. 如果不能, 那也多半是无用的内容
        _tmp_句子 = _tmp_句子[:80] + "..."
    测试文本组.append(_tmp_句子)

在做文本处理之前得看看文本到底是不是有意义的语言, 如果是, 不同语言也应该采用不同的方式对待.  
目前主流的开源自然语言语种识别工具有langdetect和langid, 其中, langdetect处理速度较高但准确率不足, langid准确率要高一些但速度也慢.  
langid目前(Jul 15, 2017)提供了97种语言的预训练, 分类出来的的语言标号依据[ISO 639-1](https://baike.baidu.com/item/ISO%20639-1)语言编码标准  

In [61]:
%%time
import langid

Wall time: 0 ns


In [66]:
%time langid.classify("喵喵喵~")  # 实际使用时发现这个分类器在第一次分类时会初始化, 有单独的较长耗时. 为了方便后面的迭代操作时间均匀, 先单独做个空运算使其初始化一下.

Wall time: 4.98 ms


('zh', -13.09517526626587)

In [41]:
项目中常见自然语言语种 = {'en':'英文','zh':'中文', 'de':'德语', 'el':'希腊语', 'ja':'日语', 'la':'拉丁语', 'ru':'俄语', 'th':'泰语'}

In [203]:
%%time
for 测试句子 in 测试文本组:
    lang, probability = langid.classify(测试句子)
    print(f"{测试句子[:80]: <{95-len(测试句子.encode('gbk'))+len(测试句子)}}\t| → {lang}({round(probability, 6)})")
# langid.classify默认输出的第二个数值是对数概率, 不计算全部语种的概率所以速度快. 但是有时候因为还需要知道语种分类的"置信度", 就需要启用langid.py的概率归一化: 

[CQ:at,id=qq/user/15*****] 氖泡接在驱动电源负和地线之间[CQ:face,id=108][CQ:face,id=108][CQ:at, 	| → zh(-204.031059)
牛啤                                                                                           	| → zh(-19.840544)
ΕΞΖΝи△M                                                                                  	| → el(-68.036244)
This is a test text...                                                                         	| → en(-54.413104)
<h2><a id="user-content-probability-normalization" class="anchor" aria-hidden="t               	| → ro(-14.682011)
import win32event as event; event_tigger = event.CreateEvent(None, False, False,               	| → en(-47.138615)
http://yhhtech.cn                                                                              	| → en(-16.985306)
2333锟斤拷                                                                                     	| → zh(-12.147441)
Wall time: 43 ms


In [63]:
%%time
from langid.langid import LanguageIdentifier, model
lider = LanguageIdentifier.from_modelstring(model, norm_probs=True) # 其实主要的耗时是出在这里

Wall time: 2.68 s


In [202]:
%%time
for 测试句子 in 测试文本组:
    lang, probability = lider.classify(测试句子)
    print(f"{测试句子[:80]: <{95-len(测试句子.encode('gbk'))+len(测试句子)}}\t| → {lang}({round(probability * 100, 2)}%)")

[CQ:at,id=qq/user/15*****] 氖泡接在驱动电源负和地线之间[CQ:face,id=108][CQ:face,id=108][CQ:at, 	| → zh(100.0%)
牛啤                                                                                           	| → zh(51.24%)
ΕΞΖΝи△M                                                                                  	| → el(93.4%)
This is a test text...                                                                         	| → en(100.0%)
<h2><a id="user-content-probability-normalization" class="anchor" aria-hidden="t               	| → ro(49.87%)
import win32event as event; event_tigger = event.CreateEvent(None, False, False,               	| → en(99.85%)
http://yhhtech.cn                                                                              	| → en(65.19%)
2333锟斤拷                                                                                     	| → zh(91.77%)
Wall time: 49 ms


还可以考虑加个编程语言探测, 比如GitHub通过[linguist](https://github.com/github/linguist)对主流编程语言(300多种)达到84%正确识别   
其实也可以自己用朴素贝叶斯实现一个, 但是暂时没时间和精力去做了...

本文以中文语境的处理为例, 使用jieba分词工具

In [59]:
%%time
import jieba as jb, jieba.analyse as jban, jieba.posseg as pseg

Wall time: 2.45 s


In [119]:
%time jb.initialize()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Hooke\AppData\Local\Temp\jieba.cache
Loading model cost 1.192 seconds.
Prefix dict has been built successfully.


Wall time: 1.2 s


jieba是一个开源的汉语分词工具, 号称做最好的python中文分词, 简便易用, 支持三种分词模式, 支持繁体, 支持[自定义词典](https://github.com/fxsjy/jieba#%E8%BD%BD%E5%85%A5%E8%AF%8D%E5%85%B8), 可以标注词性, 可以[提取文本关键词](https://github.com/fxsjy/jieba#%E5%9F%BA%E4%BA%8E-tf-idf-%E7%AE%97%E6%B3%95%E7%9A%84%E5%85%B3%E9%94%AE%E8%AF%8D%E6%8A%BD%E5%8F%96). MIT授权.    
jieba只支持中文, 英语以及类英语的语言可以用[NLTK](https://github.com/nltk/nltk)(NLTK也支持中文处理, 但使用不太友好, 中文分词还需要安装斯坦福分词器)  
jieba词性标注采用[和ictclas兼容的标记法](https://gist.github.com/luw2007/6016931)  
其他常用的汉语分词工具还有SnowNLP, PkuSeg, THULAC, HanLP 

In [120]:
汉语文本常见词性标注表 = {'d':'副词', 'vn':'动名词', 'n':'名词', 'v':'动词', 'a':'形容词', 'y':'语气词', 'c':'连词', 'x':'非语素', 'p':'介词', 'm':'数词', 'q':'量词', 'ul':'助词', 's':'处所词', 'f':'方位词', 'i':'成语', 'ns':'地名', 'o':'拟声词', 'b':'区别词', 'p':'介词', 'r':'代词', 't':'时间词', 'nr':'人名', 'l':'习用语', 'eng':'外来词', 'nz':'其他专名', 'z':'状态词'}
# 有的词性的词对于NLP来说是不重要的, 需要忽略, 比如非语素(标点符号什么的)以及语气词
不关注的词性 = ['x', 'y', 'ul', 'd']
停用词 = "啊,喔,呃,额,诶,咦,哎,嗯,哈,嘛,那啥,卧槽,艹,妈耶,话说,这个,那个,怎么,之,乎,者,也,只是,因此,以至于,差不多,哪怕,例如,尽管".split(",")

In [124]:
print(jb.lcut("雷水龙狼和冥火狐狼是一对CP, 它们都是双形态兽设, 所谓`龙狼`并不是一种设定而是龙和狼两个形态可化形."))

['雷', '水龙', '狼', '和', '冥', '火狐', '狼', '是', '一对', 'CP', ',', ' ', '它们', '都', '是', '双', '形态', '兽设', ',', ' ', '所谓', '`', '龙狼', '`', '并', '不是', '一种', '设定', '而是', '龙', '和', '狼', '两个', '形态', '可', '化形', '.']


jieba预设的词典虽然在大多时候能满足需求, 但是对于实际应用来说不够, 很多术语、“黑话”、网络口头语以及“梗”等都可能出现分词错误, 所以, 还需要根据项目实际情况自制词典. 

In [181]:
# 为了方便观察, 就把自定义词典内容写这里了...
文件_自定字典 = open('userDict.txt', 'wb', -1)  # 使用wb搭配后面的encode以便跨平台. (使用中发现在中文的Windows上通过jupyter输入文字是GB2312)
文件_自定字典.write(
'''剪脚封灌 30 vn
感应加热 600 nz
市电 200 d
那 10 c
上去 20 z
上去过 6000 vf
下去 20 z
靠 10 y
是不 5000 l
初级 20 nz
次级 20 nz
斩波 200 nz
TC 600 nz
发错 10 z
群 200 n
控制 300 vn
可行 20 z
没有卖的 z
啊 6 y
地 5000 nz
良好接地 4960 z
异常 20 nz
指定的 300 d
RAISE语句 nz
EXCEPT语句 nz
新洁能 nz
可行 40 z
MM 2333 q
616471607 nr
2139223150 nr
1837107998 nr
猫哥 nr
狗狗 nr 
龙狼 nr
特大爷 nr
本人 500 r
... x
管子 nz
营销号 nz
牛批 a
图腾柱 nz
也是 100 y
逻辑门 nz
直驱 d
M 300 q
对对对 y
来着 600 l
是啥 200 d
命名规则 nz
峰值电流 nz
MOS nz
会不会 600 d
SSTC nz
G极 nz
S极 nz
D极 nz
O叔 nr
牛逼 a
炸管 l
空穴 60 nz
射频 220 nz
鲁大师 nz
撸大师 20 nz
撸大湿 10 nz
P站 50 nz
埃及鼻涕 250 nz
氙猫 600 nr
雷水龙狼 nr
冥火狐狼 nr
我他妈 y
真他妈 y
奥利给 y
E链接转义E s
脑疾 200 nz
'''.encode("utf-8")
             )
文件_自定字典.flush(); 文件_自定字典.close()

In [182]:
%%time
jb.load_userdict('userDict.txt')
jb.initialize()

Wall time: 3 ms


In [184]:
print(jb.lcut("雷水龙狼和冥火狐狼是一对CP, 它们都是双形态兽设, 所谓`龙狼`并不是一种设定而是龙和狼两个形态可化形."))

['雷水龙狼', '和', '冥火狐狼', '是', '一对', 'CP', ',', ' ', '它们', '都', '是', '双', '形态', '兽设', ',', ' ', '所谓', '`', '龙狼', '`', '并', '不是', '一种', '设定', '而是', '龙', '和', '狼', '两个', '形态', '可', '化形', '.']


In [192]:
def 文段分句(文段):
    文段 = re.sub('([。！~ ; ； ？\?])([^”’])', r"\1\n\2", 文段)       # 单字符断句符
    文段 = re.sub('(\.{2,})([^”’])', r"\1\n\2", 文段)                  # 英文省略号
    文段 = re.sub('(\…{1,})([^”’])', r"\1\n\2", 文段)                 # 中文省略号
    文段 = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', 文段)  # 如果双引号前有终止符，那么双引号才是文段的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    文段 = 文段.rstrip()                                               # 段尾如果有多余的\n就去掉它
    return 文段.split("\n")

def 语种测试(文本):
    语种 = ""
    语种分类信息 = lider.classify(文本)
    if 语种分类信息[1] > 0.988: # 实际观察发现, 置信度低于这个值的通常是无意义内容或者不是自然语言
        try:
            语种 = 项目中常见自然语言语种[语种分类信息[0]]
        except KeyError:
            语种 = "未知"
    else:
        语种 = None
    return 语种

def 分词并标注(文本, 调试模式 = False):
    分词结果 = []; 标注结果 = []
    文本的语种 = 语种测试(文本)
    if 文本的语种 != None:
        if 调试模式 == True:
            print("###debug---检测到 %s 文本 \"%s\" 输入---" % (文本的语种, 文本) )
        if 文本的语种 == "中文":
            tmp_pseg结果 = pseg.lcut(文本)
            if 调试模式 == True:
                结果 = []
                for 词, 词性 in tmp_pseg结果:
                    try:
                        结果.append( (词, "%s(%s)" % ( 词性, 汉语文本常见词性标注表[词性]) ) )
                    except KeyError:
                        结果.append( (词, 词性) )
                return 结果
            else:
                for 词, 词性 in tmp_pseg结果:
                    if 词性 not in 不关注的词性: # 忽略多余的语素
                        分词结果.append( 词 )
                        标注结果.append( 词性 )
        else:
            raise UserWarning("暂不处理其他语种")
    else:
         raise SyntaxWarning("无法确定输入文本的语种")
    return 分词结果, 标注结果

In [188]:
分词并标注("Python可以使用raise语句抛出一个指定的异常, 之后再使用except语句根据异常信息来处理.", 调试模式 = True)

###debug---检测到 中文 文本 "Python可以使用raise语句抛出一个指定的异常, 之后再使用except语句根据异常信息来处理." 输入---


[('Python', 'eng(外来词)'),
 ('可以', 'c(连词)'),
 ('使用', 'v(动词)'),
 ('raise', 'eng(外来词)'),
 ('语句', 'n(名词)'),
 ('抛出', 'v(动词)'),
 ('一个', 'm(数词)'),
 ('指定的', 'd(副词)'),
 ('异常', 'nz(其他专名)'),
 (',', 'x(非语素)'),
 (' ', 'x(非语素)'),
 ('之后', 'f(方位词)'),
 ('再', 'd(副词)'),
 ('使用', 'v(动词)'),
 ('except', 'eng(外来词)'),
 ('语句', 'n(名词)'),
 ('根据', 'p(介词)'),
 ('异常', 'nz(其他专名)'),
 ('信息', 'n(名词)'),
 ('来', 'v(动词)'),
 ('处理', 'v(动词)'),
 ('.', 'x(非语素)')]

-----
MutBot这个项目以QQ作为主要数据来源, 不能确定消息记录里都有些什么鬼(**数据集中难免包含不和谐的敏感内容**), 所以, 整个预处理除了做自然语言语种判断、分词和词性标注之外、过滤CQ码以及多余的符号等基础操作之外, 还得做**数据清洗**(过滤掉不良信息及广告等内容)以避免不必要的麻烦以及"模型被教坏".  
这里先用使用第三方库判断内容是否有害, 然后用正则表达式简单处理CQ码/URI和系统提示.  
本项目使用的是[cherry分类器](https://github.com/Windsooon/cherry), 在此简介一下:  

> cherry分类器使用贝叶斯模型算法做分类, 还提供了混淆矩阵和ROC曲线便于分析. 开箱即用, 定制简单.  
> 分词函数支持自定义, 可定制分类算法, 可训练自己的模型  
> cherry自带两个预训练模型(没有提供数据集, 不过可以用自己的数据集训练自己的模型):  
>> * model='harmful' : 赌博 / 正常 / 政治 / 色情 (4个类别包含约 1000条 中文句子)  
>> * model='news' : 彩票 / 科技 / 财经 / 房产 / 社会 / 体育 / 娱乐 (7个 类别包含约 45000条 中文新闻)

In [195]:
%%time
import re, cherry, numpy as np

In [196]:
harmful类别 = "赌博 / 正常 / 政治 / 色情".split(" / ")

In [217]:
cherry.classify(model="harmful", text=["text须是一个list(一次多个句子效率高)"]).probability

array([[0.32062557, 0.52173518, 0.04995779, 0.10768146]])

但实际使用发现, cherry提供的预训练模型虽然对日常常见文本的分类正确率很高, 但是, 对于群聊这种场合尤其是一些技术领域, 错误率太高(简直就是不靠谱, 至少在电子爱好者群消息记录的表现实在是惨不忍睹), 即使是尝试人工加阈值和逻辑也不好使. 例如, 自带的`harmful`模型出现了以下误分情况:  

In [320]:
示例文本 = ["摩擦试的应该更难做吧", "摩擦生电效率和材料有关? 大部分能量都转化为内能", "不是只需要传送带传送电荷吗", "买个华业通信电源吧", "某位老兄的线圈就是这么死的", "GDT本身就是变压器", "最近这么多人加群", "虚拟机怎么让他全屏界面显示", "不包邮的话，白送么", "电源正负极串联电容，地线接中间，小功率接地都这样干", "服务器主板才支持热插拔", "电阻等于电压比电流", "IGBT的开关速度太低不适合上高频", "把充电头输入220接到输出", "不过说实话PD有点不敢浪,PD的CC线的耐压在之前标准上只有6V,VBUS挨着CC.", "这VSYS线上没太大电流", "开关速度不行", "cherry分类器的输入最好是批量的", "镊子插进去转两圈就好了"]

In [321]:
分类概率 = cherry.classify(model="harmful", text=示例文本).probability
for row in range(len(分类概率)):
    最大概率项 = int(np.where(分类概率[row] == 分类概率[row].max())[0]);
    if 最大概率项 != 1:
        颜色 = "\033[0;31m"
    else:
        颜色 = "\033[0;32m"
    proba = ""
    for column in range(len(分类概率[row])):
        valColor = "\033[0;33m" if column == 最大概率项 else "\033[m"
        proba += (f"{valColor}{'%0.4f' % (round(分类概率[row][column], 5))}\033[m\t")
    line = 示例文本[row][:60]; 类别 = harmful类别[最大概率项]
    print(f"句子{row:0>2}{颜色}`{line:<{76 - len(line.encode('gbk')) + len(line)}}\t`\033[m概率 → {proba:<{40}}(最大概率 → {颜色}{类别}\033[m)")

句子00`摩擦试的应该更难做吧                                                        	`概率 → 0.0381	0.0303	0.0359	0.8957	(最大概率 → 色情)
句子01`摩擦生电效率和材料有关? 大部分能量都转化为内能                              	`概率 → 0.0001	0.0135	0.9864	0.0001	(最大概率 → 政治)
句子02`不是只需要传送带传送电荷吗                                                  	`概率 → 0.5721	0.2136	0.1270	0.0873	(最大概率 → 赌博)
句子03`买个华业通信电源吧                                                          	`概率 → 0.4227	0.0453	0.4871	0.0449	(最大概率 → 政治)
句子04`某位老兄的线圈就是这么死的                                                  	`概率 → 0.1556	0.2320	0.3219	0.2905	(最大概率 → 政治)
句子05`GDT本身就是变压器                                                           	`概率 → 0.0194	0.0260	0.9151	0.0396	(最大概率 → 政治)
句子06`最近这么多人加群                                                            	`概率 → 0.0020	0.1088	0.8608	0.0284	(最大概率 → 政治)
句子07`虚拟机怎么让他全屏界面显示                                                  	`概率 → 0.0075	0.4499	0.0086	0.5340	(最大概率 → 色情)
句子08`不包邮的话，白送么                                                         

这些都是一些很正常的消息记录, 可是在错误的分了上却是得分最高, 所以必须得定制自己的模型以符合实际需求. 于是参考[cherry分类器的GitHub文档](https://github.com/Windsooon/cherry#documentation), 欲对`harmful`做扩充训练, 增`4 - 违和`和`5 - 广告`两项. 遂人工从群消息记录中选出了200条左右的各类内容存文档`data.txt`, 复制参考停用词, 建`harmfulA`文件夹, 训练.

In [193]:
import re
def CQ记录转换(原CQ记录, 调试模式 = False):
    系统提示内容 = ["&#91;闪照&#93;请使用新版手机QQ查看闪照", "&#91;视频&#93;你的QQ暂不支持查看视频短片，请升级到最新版本后查看", "&#91;QQ红包&#93;你收到一个画图红包，请升级到新版手机QQ查看", "当前版本不支持该消息类型，请使用最新版本手机QQ查看"]
    if 调试模式 == True:
        print("***debug--传入文本 \"%s\"" % 原CQ记录)
    消息_处理 = 原CQ记录
    消息 = {"CQ码":"", "内容":"", "其他数据":[]}
    tmpcounter = 0
    while tmpcounter < 5:     # 先把CQ码全都单独收起来, 然后再把URL之类的附加数据收起来, 剩下的就是内容了
        tmp = re.search('(?P<CQ码>\[CQ:\w{1,8},\w{1,10}=([\u0020-\uffff 0-9_！…]+?)(\]|\}\]))', 消息_处理) # CQ码是表情/艾特/分享之类的
        if tmp == None : break # None说明没有数据或者处理完了
        消息["CQ码"] += tmp.group(1) + ", "
        消息_处理 = 消息_处理.replace(tmp.group(1), "")
        tmpcounter += 1
    tmpcounter = 0
    while tmpcounter < 5: 
        # 考虑到实际中会有这样的消息记录: "推荐看看这篇文章: http://xxx.xxx.xxx". 其中的链接也是重要的, 但不是语素.
        tmp = re.search(r'(?P<URI>([a-z]+://)?(((([-0-9a-z一-龥\:@]+\.){1,4}(com|cn|org|tw|top|hk|info|vip|club|net|cc|me|gov|shop|wiki|([一-龥]{2,3})))|(\d+\.\d+\.\d+\.\d+))(:\d{1,5})?(\/[\u0023-\u002b\u002d-\u005a\u005f-\u9fa5=]+(\.(php|html|jpg|png|pdf|mp4|mp3|zip|rar|txt))?)?)|([a-z]+://[a-z0-9-_\=]+\/[\x21-\x7e]+))', 消息_处理)  # URI的常见形式为`[协议://][用户名[:密码]@][子域名.]顶级域名[:端口号]/资源路径[附加Querry]`, 其中方括号的内容不一定有
        # TODO: 待优化
        if tmp == None or tmp.group(1) == "" : break 
        消息["其他数据"].append(tmp)
        消息_处理 = 消息_处理.replace(tmp.group(1), "E链接转义E")
        tmpcounter += 1
    # 还有一些用不上的系统提示也需要剔除
    for element in 系统提示内容:
        消息_处理 = 消息_处理.replace(element, "")
    消息_处理 = 消息_处理.upper()  # 为了减少词复杂度和减轻计算压力, 所有字母统一大写
    # 这些步骤之后剩下的就是内容了, 考虑到内容有可能是长文本或者文段, 应该对内容分句.
    消息["内容"] = 文段分句(消息_处理)
    if 调试模式 == True:
        
    return 消息

In [194]:
CQ记录转换(
    """[CQ:at,id=qq/user/15*****] 推荐看看这篇文章:《某只狗狗春节假期不停写代码竟是因为脑疾难医》, 很有意思啊~
    还有https://h.bilibili.com/54730711 这个故事告诉了我们数据清洗的重要性.
    小女觉得吧，这几句话应当分句才是……
    某只狗狗患有脑疾, 近来愈发严重. 她每天不是对着电脑不停敲代码就是妩媚地蹲在床上娇嗲地学猫叫。所以说嘛... 不能讳疾忌医呀!
    """, 调试模式 = True)

***debug--传入文本 "[CQ:at,id=qq/user/15*****] 推荐看看这篇文章:http://yhhtech.cn/read12.html 以及 bilibili.com/video/av7078149"
{'CQ码': '[CQ:at,id=qq/user/15*****], ', '内容': [' ', '推荐看看这篇文章:E链接转义E ', '以及 ', 'E链接转义E'], '其他数据': [<re.Match object; span=(10, 39), match='http://yhhtech.cn/read12.html'>, <re.Match object; span=(20, 48), match='bilibili.com/video/av7078149'>]}


----  

以上内容基本就能实现简单的文本预处理了, 预处理数据作为后级的输入  
**目前主要的目标还是从QQ群聊天记录里提取语料和数据集**  

----

目前(05 Jan. 2020)的想法: 维护一个数据库用以处理和记录对话上下文关系以及前级(文本预处理等)的结果和处理情况

In [9]:
import sqlite3 as sqlite

In [10]:
import numpy as np, pandas as pd # 主要是为了简化原始数据的读取和预处理
pd.set_option('max_colwidth',100); pd.set_option('display.max_rows', 20); pd.set_option('display.max_columns', 8) # 为了方便查看内容

>  pandas是一款数据处理工具，集成了numpy以及matplotlib，拥有便捷的数据处理以及文件读取能力  

pandas主要有几大功能:  
1. Object Creation: pandas有三种对象, series、df和Panel Object. Series, 通过传入list对象来新建, 可以指定索引; DataFrame, 通过传入numpy数组/dictionary对象来创建.
2. Viewing Data: 查看头部/查看索引和列名/查看统计结果什么的, 还可以转换成numpy格式/做矩阵转置以及排序.
3. Selection: 类似于sql的select, 可以指定列/行/标签/值/位置/条件来筛选
4. Missing Data: 默认的空值是np.nan, 可通过reindex函数来增删改查某坐标轴(行或列)的索引，并返回一个数据的拷贝, 还可以判断是否为空值(返回False或True)
5. Operations: 一些常用计算, 包括平均值/ 数值移动等. (通过应用还可以做累计求和和其他自定义的方法)
6. Merge: pandas提供了多个方法来合并不同的对象. 其中Merge方法类似SQL的合并方式.
7. Grouping: 分组主要是为了对某些数据的计算, 例如df.groupby('A').sum()
8. Reshaping
9. Time Series: pandas很适合用来处理时序, 可以调整时间间隔/时区转换/时间格式转换
10. [Categoricals](http://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categorical)
11. Plotting: 用于数据绘图, 和Matplotlib基本一样
12. Getting Data In/Out : 可以方便地在不同类型的文件(csv/text/json/html/excel/sql等)中导入或导出数据

完整的[user guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/)可以在pydata上看到  
*这里主要是要用pandas的DataFrame简化sql操作*  
用 pandas DataFrame 读取数据结果的好处主要是不需要每次都调用 fetchall之类的函数, 还能能方便地通过表头的名字来阅读整个表

In [11]:
DB_CQ = sqlite.connect("eventv2.db")
DB_APP = sqlite.connect("app.db")
# 出于尊重隐私考虑, 不提供这两个数据库, 并且这两个数据库相关的部分内容打了码

In [12]:
计时_单元格开始 = time.time()
query_selectLog = "SELECT `id`, `tag`, `GROUP`, `account`, `operator`, `content`, `TIME` FROM `event` ORDER by `id` DESC;"
DF_CQdata = pd.read_sql_query(query_selectLog, DB_CQ)
处理耗时 = (time.time() - 计时_单元格开始) * 1000
print("debug --- 处理本单元格耗时 %0.3f 毫秒" % 处理耗时 )
print("debug --- 对象`DF_CQdata`占用内存 → %0.3fKB" % (sys.getsizeof(DF_CQdata) / 1000)  )

debug --- 处理本单元格耗时 237.310 毫秒
debug --- 对象`DF_CQdata`占用内存 → 28692.089KB


In [13]:
DF_CQdata

,id,tag,group,account,operator,content,time
0,71661,contact,qq/group/569611113,qq/user/2467551879,,有人吗,1580614764
1,71660,contact,qq/group/569611113,qq/user/1252487847,,"[CQ:face,id=178][CQ:face,id=178]",1580614648
2,71659,contact,qq/group/569611113,qq/user/3354453079,,"[CQ:face,id=178]",1580614580
3,71658,contact,qq/group/569611113,qq/user/3354453079,,66,1580614576
4,71657,contact,qq/group/569611113,qq/user/3514532587,,"[CQ:at,id=qq/user/3354453079] [CQ:at,id=qq/user/3354453079] 没",1580614567
...,...,...,...,...,...,...,...
71504,5,,,qq/user/2139223150,,来自616471607的私聊消息(4):,1577880261
71505,4,contact,,qq/user/616471607,,29,1577880261
71506,3,contact,qq/group/569611113,qq/user/1528442205,,像我这种小功率都内部接地,1577880243
71507,2,contact,qq/group/569611113,qq/user/513877092,,"[CQ:at,id=qq/user/1426259809] [CQ:at,id=qq/user/1426259809] 我就谁便用了啵",1577880188


In [14]:
计时_单元格开始 = time.time()
经过预处理的消息 = {} 
tmp_CQ转换 = ""
tmp_分词标注 = ""
time_start = time.time()
for row in DF_CQdata.itertuples(): 
# 用.iterrows()虽然可读性更好, 但是它是真的慢...用.iterrows()的话在小米9上耗时差不多都是600秒以上...所以还是改用.itertuples()
    tmp_CQ转换 = CQ记录转换(row[6])
    if len(tmp_CQ转换["内容"]) > 0:
        try:
            tmp_分词标注 = 分词并标注(tmp_CQ转换["内容"]) 
        except UserWarning:
            tmp_分词标注 = ["[\u2333#][*unkownLanguage*] " + tmp_CQ转换["内容"], []]
        except SyntaxWarning:
            tmp_分词标注 = ["[\u2333#][*unclearContent*] " + tmp_CQ转换["内容"], []]
    else:
        tmp_分词标注= [[],[]]
    经过预处理的消息[row[1]] = {'CQ码':tmp_CQ转换['CQ码'] if len(tmp_CQ转换['CQ码']) > 0 else "", '分词标注':[tmp_分词标注[0], tmp_分词标注[1]] if len(tmp_分词标注[0]) > 0 else [[], []]}
    
print("文本预处理了 %d 条记录. 耗时→%.3f秒" % (len(经过预处理的消息), time.time() - time_start) )
tmp_对照变量 = "abcde12345"
print("debug --- 变量`tmp_对照变量`占用内存 → %0.3fKB" % (sys.getsizeof(tmp_对照变量) / 1000) )
print("debug --- 变量`经过预处理的消息`占用内存 → %0.3fKB" % (sys.getsizeof(经过预处理的消息) / 1000) )

tmp_循环限制计次 = 0
for 消息ID, 预处理结果 in 经过预处理的消息.items():
    tmp_循环限制计次 += 1
    print("%d: CQ码→%s | 分词标注→%s" %(消息ID, 预处理结果['CQ码'] if len(预处理结果['CQ码']) > 0 else "无", 预处理结果['分词标注'] if len(预处理结果['分词标注'][0]) > 0 else "无" ) )
    if tmp_循环限制计次 > 35: break
处理耗时 = (time.time() - 计时_单元格开始) * 1000
print("debug --- 处理本单元格耗时 %0.3f 毫秒" % 处理耗时 )

文本预处理了 71509 条记录. 耗时→253.739秒
debug --- 变量`tmp_对照变量`占用内存 → 0.059KB
debug --- 变量`经过预处理的消息`占用内存 → 2621.552KB
71661: CQ码→无 | 分词标注→[['有人'], ['r']]
71660: CQ码→[CQ:face,id=178],  | 分词标注→无
71659: CQ码→[CQ:face,id=178],  | 分词标注→无
71658: CQ码→无 | 分词标注→['[⌳#][*unclearContent*] 66', []]
71657: CQ码→[CQ:at,id=qq/user/3354453079],  | 分词标注→['[⌳#][*unclearContent*]   没', []]
71656: CQ码→[CQ:rich,title=&#91;QQ小程序&#93;哔哩哔哩,content={"detail_1":{"appid":"1109937557"&#44;"desc":"特斯拉线圈入门教程-电路部分讲解"&#44;"host":{"nick":"晨夕"&#44;"uin":1143297121}&#44;"icon":"miniapp.gtimg.cn/public/appicon/432b76be3a548fc128acaa6c1ec90131_200.jpg"&#44;"preview":"pubminishare-30161.picsz.qpic.cn/9fb1c137-bffd-44ac-bae2-eedd2a7248a5"&#44;"scene":1036&#44;"shareTemplateData":{}&#44;"shareTemplateId":"8C8E89B49BE609866298ADDFF2DBABA4"&#44;"title":"哔哩哔哩"&#44;"url":"m.q.qq.com/a/s/317427058ccfe57478b11c4be7c65ebb"}}],  | 分词标注→无
71655: CQ码→[CQ:rich,title=&#91;QQ小程序&#93;哔哩哔哩,content={"detail_1":{"appid":"1109937557"&#44;"desc":"特斯拉线圈入门教程-

In [15]:
# 再维护一个数据库用以记录预处理过的内容以及上下文关系
DB_ppced = r"CQevePPC.db"
Conn_ppced = sqlite.connect(DB_ppced)

目前(10 Jan. 2020)的想法是使用两个表: `ppcLog` 和 `contextIndex`, 分别用于记录预处理后的数据和上下文的关系(按群和时间分组, 以便后续提取对话)

In [16]:
# 数据库设计
query_createTable_ppcLog = '''
CREATE TABLE "ppcLog" (
"lid"       INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
"Link"      INTEGER,
"cqCode"    TEXT,
"jiebaCUT"  TEXT,
"jiebaPSEG" TEXT,
"lastModif" NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
"note"      TEXT,
"extraData" BLOB
);'''
query_createTable_contextIndex = '''
CREATE TABLE "contextIndex" (
"id_conversation"   INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
"GroupFrom"         INTEGER,
"UserFrom"          INTEGER,
"dialogsContent"    TEXT,
"DialogsLink"       INTEGER
"time_sort"         NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
"note"              TEXT
);'''

In [17]:
# 先检查数据库状态, 如果没有预设的表那就初始化
def DBtest(dbConnHandle, tableNames, checkQuery = []):
    querys_tableCheck = []
    for i in range( len(tableNames) ):
        querys_tableCheck.append("SELECT * FROM `sqlite_master` where type = 'table' and name = '%s';" % tableNames[i])
    DBisOK = False
    OrigSct = ""
    tmp_cont = 0
    cont_table = 0
    cont_unexc = 0
    for query in querys_tableCheck:
        curs = dbConnHandle.execute(query)
        gets = curs.fetchall()
        if len(gets) > 0:
            OrigSct = gets[-1][-1]
            print("数据表`%s`存在. ~ 读到记录:  \n%s " % (tableNames[tmp_cont], OrigSct) )
            cont_table += 1
            if len(checkQuery) == len(tableNames):
                if OrigSct.replace("\n", "").replace(" ","").replace(";", "").replace("\t", "") != checkQuery[tmp_cont].replace("\n", "").replace(" ","").replace(";", "").replace("\t", ""):
                    cont_unexc -= 4
                else:
                    DBisOK += 1
        tmp_cont += 1

    if checkQuery != []:
        if cont_table < 1 :
            print("指定的表不存在")
            DBisOK = False
        if cont_unexc < 0: 
            if cont_table > 0:
                raise SyntaxWarning("数据库检查未通过: 已经存在不同结构的同名表. (有 %d 个表记录与提供的记录不一致)" % (-cont_unexc / 4))
                return cont_unexc
        else:
            if DBisOK == len(tableNames) :
                print("数据库检查通过. %d 个表(共%d个表)校验一致" % (DBisOK, len(tableNames)) )
                DBisOK = True    
    else:
        if DBisOK == len(tableNames) :
            print("指定表存在")
            DBisOK = True
        else:
            print("指定表不存在")
            DBisOK = False
    return DBisOK        

In [18]:
def DB_init(dbConnHandle, tableNames = [], checkQuery = []):
    realyNeedInit = False
    try:
        dbStatus = DBtest(dbConnHandle, tableNames, checkQuery)
    except SyntaxWarning as info:
        print("初始化失败, 建议检查运行环境(文件冲突). 错误信息:  \n    ", info)
        return -233
    if dbStatus == False:
        realyNeedInit = True
    elif dbStatus < 0:
        r
    if realyNeedInit == True:
        print("准备开始初始化...")
        try:
            for query in checkQuery:
                dbConnHandle.execute(query)
            print("提交变动...")
            dbConnHandle.commit()
            print("数据表初始化完成")
        except Exception as errinfo: 
            print("---SQL执行失败---: ", errinfo)
    else:
        print("已存在同名表, 跳过初始化")

In [19]:
DB_init(Conn_ppced, ["ppcLog", "contextIndex"], [query_createTable_ppcLog, query_createTable_contextIndex])

指定的表不存在
准备开始初始化...
提交变动...
数据表初始化完成


In [20]:
DBtest(Conn_ppced, ["ppcLog", "contextIndex"], [query_createTable_ppcLog, query_createTable_contextIndex])

数据表`ppcLog`存在. ~ 读到记录:  
CREATE TABLE "ppcLog" (
"lid"       INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
"Link"      INTEGER,
"cqCode"    TEXT,
"jiebaCUT"  TEXT,
"jiebaPSEG" TEXT,
"lastModif" NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
"note"      TEXT,
"extraData" BLOB
) 
数据表`contextIndex`存在. ~ 读到记录:  
CREATE TABLE "contextIndex" (
"id_conversation"   INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
"GroupFrom"         INTEGER,
"UserFrom"          INTEGER,
"dialogsContent"    TEXT,
"DialogsLink"       INTEGER
"time_sort"         NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
"note"              TEXT
) 
数据库检查通过. 2 个表(共2个表)校验一致


True

In [21]:
# 接下来是把预处理好的内容存入数据库, 并标注哪些内容已经处理以及修改时间

# 为了提高插入数据的效率, 使用"executemany". 当然, 还可以用pandas.DataFrame.to_sql, 不过这里就怎么省事怎么来了.  
# 需要注意的是, 如果想要用sqlite3的python接口"executemany", 传入数据需要是`[(xxx,xx,...,x),...(x,xxx,...,xx)]`的结构, 其中每个元组的元素数要对应占位符. 另外, 一次处理的量不能太大.
  
Conn_ppced.executemany(
    "insert into ppcLog (`Link`, `cqCode`, `jiebaCUT`, `jiebaPSEG`, `note`) values (?, ?, ?, ?, 'initialize')",
    [ ( link, res['CQ码'], str(res["分词标注"][0]), str(res["分词标注"][1]) )  for link, res in 经过预处理的消息.items() ]
)
Conn_ppced.commit() # 操作后应该尽快提交, 以免数据库锁带来的麻烦(尤其是多线程)

In [22]:
# 提交完了, 瞅瞅情况
query_selectLog = "SELECT * FROM `ppcLog`;"
数据库中的预处理结果 = pd.read_sql_query(query_selectLog, Conn_ppced)

In [23]:
数据库中的预处理结果

,lid,Link,cqCode,jiebaCUT,jiebaPSEG,lastModif,note,extraData
0,1,71661,,['有人'],['r'],2020-02-02 12:31:04,initialize,None
1,2,71660,"[CQ:face,id=178],",[],[],2020-02-02 12:31:04,initialize,None
2,3,71659,"[CQ:face,id=178],",[],[],2020-02-02 12:31:04,initialize,None
3,4,71658,,[⌳#][*unclearContent*] 66,[],2020-02-02 12:31:04,initialize,None
4,5,71657,"[CQ:at,id=qq/user/3354453079],",[⌳#][*unclearContent*] 没,[],2020-02-02 12:31:04,initialize,None
...,...,...,...,...,...,...,...,...
71504,71505,5,,"['来自', '616471607', '的', '私聊', '消息']","['v', 'nr', 'uj', 'a', 'n']",2020-02-02 12:31:04,initialize,None
71505,71506,4,,[⌳#][*unclearContent*] 29,[],2020-02-02 12:31:04,initialize,None
71506,71507,3,,"['像', '我', '这种', '小', '功率', '都', '内部', '接地']","['v', 'r', 'r', 'a', 'n', 'd', 'f', 'v']",2020-02-02 12:31:04,initialize,None
71507,71508,2,"[CQ:at,id=qq/user/1426259809],","['我', '就', '谁', '便用', '了']","['r', 'd', 'r', 'v', 'ul']",2020-02-02 12:31:04,initialize,None


-----

### 以上内容便实现了数据接口以及数据预处理的基础功能(所以作为第0部分), 接下来开始真正的NLP内容  

本文内容仅为个人见解和心得, 希望能带来帮助. 如果有不正确/不准确之处还请多多指教, 一起学习一起进步~   

本文内容由[佚之狗](https://github.com/HookeLiu)原创, 可以随意使用但请注明出处.    